Homework Week 3 - Spark Fundamentals

Start by loading the datasets from the iceberg data folder. The queries will use the following tables:
- match_details
- matches
- medals_matches_players
- medals
- maps
<br> Additional files include:<br>
 - devices
 - events

In [31]:
events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn('event_date', expr("DATE_TRUNC('day', event_time)"))

events.show()

+-----------+----------+--------+--------------------+----------+--------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|
+-----------+----------+--------+--------------------+----------+--------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|
| -824540328| 532630305|    NULL|admin.zachwilson....|         /|2021-09-28 17:23:...|
| -824540328| 532630305|    NULL|    www.eczachly.com|         /|2021-09-29 01:22:...|
| 1833036683| 532630305|    NULL|admin.zach

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv").withColumn('partitioned_date', expr("DATE_TRUNC('day', event_time)"))
matches.show()

+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+-------------------+
|            match_id|               mapid|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|      map_variant_id|   partitioned_date|
+--------------------+--------------------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------------+-------------------+
|11de1a94-8d07-416...|c7edbf0f-f206-11e...|        true|f72e0ef0-7c4a-430...|1e473914-46e4-408...|         true|2016-02-22 00:00:...|          NULL|     NULL|                NULL|2016-02-22 00:00:00|
|d3643e71-3e51-43e...|cb914b9e-f206-11e...|       false|d0766624-dbd7-453...|257a305e-4dd3-41f...|         true|2016-02-14 00:00:...|          NULL|     NULL|                NULL|2016-02-14 00:00:00|


In [11]:
match_details = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")

match.show()

+--------------------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+
|            match_id|player_gamertag|previous_spartan_rank|spartan_rank|previous_total_xp|total_xp|previous_csr_tier|previous_csr_designation|previous_csr|previous_csr_percent_to_next_tier|previous_csr_rank|current_

In [74]:
null_count = match_details.filter(match_details.team_id.isNull()).count()
null_count

11

In [13]:
medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")
medals_matches_players.show()

+--------------------+---------------+----------+-----+
|            match_id|player_gamertag|  medal_id|count|
+--------------------+---------------+----------+-----+
|009fdac5-e15c-47c...|       EcZachly|3261908037|    7|
|009fdac5-e15c-47c...|       EcZachly| 824733727|    2|
|009fdac5-e15c-47c...|       EcZachly|2078758684|    2|
|009fdac5-e15c-47c...|       EcZachly|2782465081|    2|
|9169d1a3-955c-4ea...|       EcZachly|3001183151|    1|
|9169d1a3-955c-4ea...|       EcZachly|3565443938|    6|
|9169d1a3-955c-4ea...|       EcZachly|3491849182|    1|
|4a078b2f-65eb-4c6...|       EcZachly|3261908037|    8|
|9169d1a3-955c-4ea...|       EcZachly|2105198095|    6|
|9169d1a3-955c-4ea...|       EcZachly|2916014239|    3|
|9169d1a3-955c-4ea...|       EcZachly|3261908037|    6|
|9169d1a3-955c-4ea...|       EcZachly|1351381581|    2|
|9169d1a3-955c-4ea...|       EcZachly|2838259753|    1|
|9169d1a3-955c-4ea...|       EcZachly|3354395650|    1|
|9169d1a3-955c-4ea...|       EcZachly| 298813630

In [14]:
medals = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")
medals.show()

+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|  medal_id|          sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|   classification|         description|          name|difficulty|
+----------+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+--------------+----------+
|2315448068|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|3565441934|                NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|             NULL|                NULL|          NULL|      NULL|
|4162659350|https://content.h...|        750|       750|    

Goal:
- Build a disabled automatic broadcast join with spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
- Explicitly broadcast JOINs medals and maps
- Bucket join match_details, matches, and medal_matches_players on match_id with 16 buckets
- Aggregate the joined data frame to determine the following: 
    - Player with highest kills per game average
    - Playlist with most plays
    - Map most played
    - Map with most players earning Killing Spree medals
- With the aggregated dataset, try different .sortWithinPartitions to see which has the smallest data size (check playlists and maps)

Next, I need to create a database and insert the data from these tables so we can use SQL to query the data

In [15]:
%%sql
CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [79]:
%%sql
DROP TABLE IF EXISTS bootcamp.match_details

++
||
++
++

In [62]:
%%sql
DROP TABLE IF EXISTS bootcamp.matches

++
||
++
++

In [63]:
%%sql
DROP TABLE IF EXISTS bootcamp.medals_matches_players

++
||
++
++

In [64]:
%%sql
DROP TABLE IF EXISTS bootcamp.medals

++
||
++
++

In [65]:
%%sql
DROP TABLE IF EXISTS bootcamp.maps

++
||
++
++

In [24]:
match_details.dtypes

[('match_id', 'string'),
 ('player_gamertag', 'string'),
 ('previous_spartan_rank', 'string'),
 ('spartan_rank', 'string'),
 ('previous_total_xp', 'string'),
 ('total_xp', 'string'),
 ('previous_csr_tier', 'string'),
 ('previous_csr_designation', 'string'),
 ('previous_csr', 'string'),
 ('previous_csr_percent_to_next_tier', 'string'),
 ('previous_csr_rank', 'string'),
 ('current_csr_tier', 'string'),
 ('current_csr_designation', 'string'),
 ('current_csr', 'string'),
 ('current_csr_percent_to_next_tier', 'string'),
 ('current_csr_rank', 'string'),
 ('player_rank_on_team', 'string'),
 ('player_finished', 'string'),
 ('player_average_life', 'string'),
 ('player_total_kills', 'string'),
 ('player_total_headshots', 'string'),
 ('player_total_weapon_damage', 'string'),
 ('player_total_shots_landed', 'string'),
 ('player_total_melee_kills', 'string'),
 ('player_total_melee_damage', 'string'),
 ('player_total_assassinations', 'string'),
 ('player_total_ground_pound_kills', 'string'),
 ('playe

In [77]:
from pyspark.sql.functions import countDistinct
unique_players_count = match_details.select(countDistinct('player_gamertag')).collect()[0][0]

unique_players_count

69420

In [78]:
unique_teams = match_details.select(countDistinct('team_id')).collect()[0][0]
unique_teams

8

In [76]:
unique_matches = match_details.select(countDistinct('match_id')).collect()[0][0]
unique_matches

19050

In [71]:
match_details.count()

151761

In [81]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.match_details (
    match_id STRING,
    player_gamertag STRING,
    previous_spartan_rank INT,
    spartan_rank INT,
    previous_total_xp BIGINT,
    total_xp BIGINT,
    previous_csr_tier INT,
    previous_csr_designation INT,
    previous_csr INT,
    previous_csr_percent_to_next_tier DOUBLE,
    previous_csr_rank INT,
    current_csr_tier INT,
    current_csr_designation INT,
    current_csr INT,
    current_csr_percent_to_next_tier DOUBLE,
    current_csr_rank INT,
    player_rank_on_team INT,
    player_finished BOOLEAN,
    player_average_life STRING,
    player_total_kills INT,
    player_total_headshots INT,
    player_total_weapon_damage DOUBLE,
    player_total_shots_landed INT,
    player_total_melee_kills INT,
    player_total_melee_damage DOUBLE,
    player_total_assassinations INT,
    player_total_ground_pound_kills INT,
    player_total_shoulder_bash_kills INT,
    player_total_grenade_damage DOUBLE,
    player_total_power_weapon_damage DOUBLE,
    player_total_power_weapon_grabs INT,
    player_total_deaths INT,
    player_total_assists INT,
    player_total_grenade_kills INT,
    did_win BOOLEAN,
    team_id INT
  ) USING iceberg PARTITIONED BY (team_id)

++
||
++
++

In [26]:
matches.dtypes

[('match_id', 'string'),
 ('mapid', 'string'),
 ('is_team_game', 'string'),
 ('playlist_id', 'string'),
 ('game_variant_id', 'string'),
 ('is_match_over', 'string'),
 ('completion_date', 'string'),
 ('match_duration', 'string'),
 ('game_mode', 'string'),
 ('map_variant_id', 'string')]

In [69]:
%%sql 
CREATE TABLE
  IF NOT EXISTS bootcamp.matches (
    match_id STRING,
    mapid STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    game_variant_id STRING,
    is_match_over BOOLEAN,
    completion_date TIMESTAMP,
    match_duration STRING,
    game_mode STRING,
    map_variant_id STRING,
    partitioned_date DATE
  ) USING iceberg PARTITIONED BY (mapid)

++
||
++
++

In [40]:
medals_matches_players.dtypes

[('match_id', 'string'),
 ('player_gamertag', 'string'),
 ('medal_id', 'string'),
 ('count', 'string')]

In [68]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.medals_matches_players (
    match_id STRING,
    player_gamertag STRING,
    medal_id BIGINT,
    COUNT INT
  ) USING iceberg PARTITIONED BY (medal_id)

++
||
++
++

In [86]:
medals_matches_players.count()

755229

In [44]:
medals.dtypes

[('medal_id', 'string'),
 ('sprite_uri', 'string'),
 ('sprite_left', 'string'),
 ('sprite_top', 'string'),
 ('sprite_sheet_width', 'string'),
 ('sprite_sheet_height', 'string'),
 ('sprite_width', 'string'),
 ('sprite_height', 'string'),
 ('classification', 'string'),
 ('description', 'string'),
 ('name', 'string'),
 ('difficulty', 'string')]

In [85]:
medals.count()

183

In [88]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.medals (
    medal_id BIGINT,
    sprite_uri STRING,
    sprite_left INT,
    sprite_top INT,
    sprite_sheet_width INT,
    sprite_sheet_height INT,
    sprite_width INT,
    sprite_height INT,
    classification STRING,
    description STRING,
    name STRING,
    difficulty INT
  ) USING iceberg

++
||
++
++

In [53]:
maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
maps.show()

+--------------------+-------------------+--------------------+
|               mapid|               name|         description|
+--------------------+-------------------+--------------------+
|c93d708f-f206-11e...|              Urban|Andesia was the c...|
|cb251c51-f206-11e...|     Raid on Apex 7|This unbroken rin...|
|c854e54f-f206-11e...|March on Stormbreak|                NULL|
|c8d69870-f206-11e...| Escape from A.R.C.|Scientists flocke...|
|73ed1fd0-45e5-4bb...|             Osiris|                NULL|
|96c3e3dd-7703-408...|          Blue Team|                NULL|
|1c4f8e19-b046-4f7...|            Glassed|                NULL|
|825065cf-df57-42e...|        Unconfirmed|                NULL|
|9a188f67-1664-4d7...|           Alliance|                NULL|
|2702ea83-2c3e-4fd...|   Before the Storm|                NULL|
|82f8471c-a2ef-408...|            Genesis|                NULL|
|fcd7caa4-37c9-436...|       The Breaking|                NULL|
|7dc80b62-dd39-41d...|          Guardian

In [58]:
maps.count()

40

In [66]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.maps (mapid STRING, name STRING, description STRING) USING iceberg

++
||
++
++

## Prior to making any joins, I need to set spart such that it doesn't automatically broadcast joins

In [83]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [84]:
print(spark.conf.get("spark.sql.autoBroadcastJoinThreshold"))

-1


Write the data to the database

In [89]:
maps.write.mode("overwrite").saveAsTable("bootcamp.maps")
medals.write.mode("overwrite").saveAsTable("bootcamp.medals")

In [92]:
%%sql
SELECT
  *
FROM
  bootcamp.maps
LIMIT
  5

mapid,name,description
c93d708f-f206-11e4-a815-24be05e24f7e,Urban,Andesia was the crucible for countless heroes and villains caught in the throes of seething rebellion and righteous excess.
cb251c51-f206-11e4-8541-24be05e24f7e,Raid on Apex 7,This unbroken ring is a symbol of the discovery that shook the galaxy and changed the course of both human and Covenant destiny.
c854e54f-f206-11e4-bddc-24be05e24f7e,March on Stormbreak,None
c8d69870-f206-11e4-b477-24be05e24f7e,Escape from A.R.C.,Scientists flocked to this Forerunner excavation in search of new beginnings. What they unearthed will lead to their inevitable end.
73ed1fd0-45e5-4bb9-ab6a-d2852c04ea7d,Osiris,None


In [93]:
%%sql
SELECT
  *
FROM
  bootcamp.medals
LIMIT
  5

medal_id,sprite_uri,sprite_left,sprite_top,sprite_sheet_width,sprite_sheet_height,sprite_width,sprite_height,classification,description,name,difficulty
2315448068,None,None,None,None,None,None,None,None,None,None,None
3565441934,None,None,None,None,None,None,None,None,None,None,None
4162659350,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,750,750,74,74,1125,899,Breakout,Kill the last enemy within the last 10 seconds of a round.,Buzzer Beater,45
1573153198,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,0,300,74,74,1125,899,Breakout,Survive a one-on-one encounter.,Vanquisher,30
298813630,https://content.halocdn.com/media/Default/games/halo-5-guardians/sprites/medalspritesheet-be288ea5c0994a4e9d36f43aee7bc631.png,0,825,74,74,1125,899,Style,Kill an enemy with Spartan Charge.,Spartan Charge,135


In [94]:
matches.write.mode("overwrite").saveAsTable("bootcamp.matches")
match_details.write.mode("overwrite").saveAsTable("bootcamp.match_details")
medals_matches_players.write.mode("overwrite").saveAsTable("bootcamp.medals_matches_players")

To explicitly broadcast the table joins for the medals and maps, we need to have an understanding of the tables: 

match_details ─┬─── matches ─── maps (via map_id)
               └─── medals_matches_players
                         │
                         └─── medals

Fact Tables: 
- match_details
- medals_matches_players
- matches

Dimension Tables: 
- medals
- maps

Prior to joining, we also need to consider the possibility of Duplicates, and dedupe the medals and maps tables (as well as the others)

In [99]:
duplicate_maps = maps.groupBy(maps.columns).count().filter("count >1")
duplicate_maps.show()

+-----+----+-----------+-----+
|mapid|name|description|count|
+-----+----+-----------+-----+
+-----+----+-----------+-----+



In [101]:
duplicate_medals = medals.groupBy(medals.columns).count().filter("count > 1")
duplicate_medals.show()

+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----+----------+-----+
|medal_id|sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|classification|description|name|difficulty|count|
+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----+----------+-----+
+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----+----------+-----+



In [102]:
duplicate_match_details = match_details.groupBy(match_details.columns).count().filter("count > 1")
duplicate_match_details.show()

+--------+---------------+---------------------+------------+-----------------+--------+-----------------+------------------------+------------+---------------------------------+-----------------+----------------+-----------------------+-----------+--------------------------------+----------------+-------------------+---------------+-------------------+------------------+----------------------+--------------------------+-------------------------+------------------------+-------------------------+---------------------------+-------------------------------+--------------------------------+---------------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------------+-------+-------+-----+
|match_id|player_gamertag|previous_spartan_rank|spartan_rank|previous_total_xp|total_xp|previous_csr_tier|previous_csr_designation|previous_csr|previous_csr_percent_to_next_tier|previous_csr_rank|current_csr_tier|current_c

In [115]:
from pyspark.sql.functions import sum, avg, max, min, col, lit, count

duplicate_medals_matches_players = medals_matches_players.groupBy("match_id", "player_gamertag", "medal_id") \
    .agg(count("*").alias("duplicate_count")) \
    .filter("duplicate_count > 1")

duplicate_medals_matches_players.show()

+--------+---------------+--------+---------------+
|match_id|player_gamertag|medal_id|duplicate_count|
+--------+---------------+--------+---------------+
+--------+---------------+--------+---------------+



In [116]:
duplicate_matches = matches.groupBy(matches.columns).count().filter("count > 1")
duplicate_matches.show()

+--------+-----+------------+-----------+---------------+-------------+---------------+--------------+---------+--------------+----------------+-----+
|match_id|mapid|is_team_game|playlist_id|game_variant_id|is_match_over|completion_date|match_duration|game_mode|map_variant_id|partitioned_date|count|
+--------+-----+------------+-----------+---------------+-------------+---------------+--------------+---------+--------------+----------------+-----+
+--------+-----+------------+-----------+---------------+-------------+---------------+--------------+---------+--------------+----------------+-----+



There do not seem to be any duplicate rows in any of the tables. 

In [1]:
from pyspark.sql.functions import broadcast, split, lit


The next step is to join all of the tables. 
- Maps and medals will be broadcasted explicitly
- the fact tables will be bucketed into 16 buckets
- the fact tables will be joined on match_id

In [3]:
# Bucket the fact tables

matches_bucketed = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/data/matches.csv")

match_details_bucketed = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/data/match_details.csv")

medals_matches_players_bucketed = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/data/medals_matches_players.csv")

In [4]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [5]:
%%sql 
CREATE DATABASE IF NOT EXISTS bootcamp

24/12/11 21:50:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [7]:
# Configure Spark catalog for local storage
spark.conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.iceberg.catalog.SparkCatalog")
spark.conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
spark.conf.set("spark.sql.catalog.spark_catalog.warehouse", "file:///tmp/warehouse")

In [11]:
# Create external table
bucketed_matches_ddl = """
CREATE EXTERNAL TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    completion_date TIMESTAMP
)
CLUSTERED BY (match_id) INTO 16 BUCKETS
STORED AS parquet
LOCATION '/tmp/external/matches_bucketed'
"""

spark.sql(bucketed_matches_ddl)

Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/bootcamp/matches_bucketed/metadata/00000-fad46569-9160-441f-9d16-84baae194736.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:348)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:331)
	at org.apache.iceberg

In [ ]:
matchesBucketed.select(
     $"match_id", $"is_team_game", $"playlist_id", $"completion_date"
     )
     .write.mode("append")
     .partitionBy("completion_date")
     .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")

In [ ]:
matchesBucketed.createOrReplaceTempView("matches")

In [12]:
# First, read the CSV data
matches_df = spark.read.option("header", "true") \
                      .option("inferSchema", "true") \
                      .csv("/home/iceberg/data/matches.csv")

# Create a temporary view
matches_df.createOrReplaceTempView("matches_temp")

# Create the bucketed table using CTAS (Create Table As Select)
create_table_sql = """
CREATE TABLE bootcamp.matches_bucketed
USING parquet
CLUSTERED BY (match_id) INTO 16 BUCKETS
AS SELECT match_id, is_team_game, playlist_id, completion_date 
FROM matches_temp
"""

# Drop the table if it exists
spark.sql("DROP TABLE IF EXISTS bootcamp.matches_bucketed")

# Create the new table
spark.sql(create_table_sql)

Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/bootcamp/matches_bucketed/metadata/00000-fad46569-9160-441f-9d16-84baae194736.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:348)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:331)
	at org.apache.iceberg